In [1]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from keras.callbacks import TensorBoard
from IPython.display import clear_output
import matplotlib.pyplot as plt
import os
import datetime
import pygame


physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
    

c:\Users\robin\anaconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\robin\anaconda3\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure yo

# Proximal Policy Optimization

PPO is a policy gradient Actor-Critic algorithm. The policy model, the **actor** network  produces a stochastic policy. It maps the state to a probability distribution over the set of possible actions. The **critic** network is used to approximate the value function and then, the advantage is calculated:

$$
A_\Phi (s_t, a_t) = q_\Phi (s_t,a_t) - v_\Phi (s_t) = R_t + \gamma v_{\Phi'} (s_{t+1}) - v_\Phi (s_t)
$$

The critic, $v_\Phi$ is trained in the same manner, as the DQN model and the critic of DDPG, with TD-learning and a "frozen" and periodically updated target critic network, $v_{\Phi'}$. Instead of approximating a q-value, it approximates the value.

To train the actor, PPO uses the ratio of two policies:
- a current policy $\pi_\Theta$, that is learned currently
- a baseline policy $\pi_{\Theta´}$, an earlier version of the policy

$$
r^t (\Theta)=r_\Theta (s_t,a_t) = \frac{\pi_\Theta (a_t | s_t)}{\pi_{\Theta'} (a_t | s_t)}
$$

It is the ratio of the probabilities of selecting $a_t$ given $\pi_\Theta$ and the probability of selecting the same action with $\pi_{\Theta´}$.

When multiplied with the the approximated advantage, calculated using the critic network, it can be used as the objective function (maximize with SGA)

$$
loss_{actor} = - r_\Theta (s_t, a_t) A_\Phi (s_t, a_t)
$$

as when
- the advantage is positive, meaning, that selecting the action would increase the value, the probability of selecting this action would increase
- the advantage is negative, meaning, that selecting the action would decrease the value, the probability of selecting this action would decrease

Instead of using this directly as loss function, to stabilize the implementation by adjusting the policy optimization step size, the loss is extended in a pessimistic way:

$$
loss_{actor} = \min [r_\Theta (s_t, a_t) A_\Phi (s_t, a_t), clip(r_\Theta (s_t, a_t), 1-\epsilon, 1+\epsilon) A_\Phi (s_t, a_t)]
$$

PPO uses 2 main models. The actor network learns the stochastic policy. It maps the state to a probability distribution over the set of possible actions. The critic network learns the value function. It maps the state to a scalar.

The critic, $v_\Phi$ is trained in the same manner, as the DQN model and the critic of DDPG, with TD-learning and a "frozen" and periodically updated target critic network, $v_{\Phi'}$. Instead of approximating a q-value, it approximates the value.

To train the actor, PPO uses the ratio of two policies:
- a current policy $\pi_\Theta$, that is learned currently
- a baseline policy $\pi_{\Theta´}$, an earlier version of the policy

$$
r^t (\Theta)=r_\Theta (s_t,a_t) = \frac{\pi_\Theta (a_t | s_t)}{\pi_{\Theta'} (a_t | s_t)}
$$

It is the ratio of the probabilities of selecting $a_t$ given $\pi_\Theta$ and the probability of selecting the same action with $\pi_{\Theta´}$.

When multiplied with the the approximated advantage, calculated using the critic network, it can be used as the objective function (maximize with SGA)

$$
loss_{actor} = - r_\Theta (s_t, a_t) A_\Phi (s_t, a_t)
$$

as when
- the advantage is positive, meaning, that selecting the action would increase the value, the probability of selecting this action would increase
- the advantage is negative, meaning, that selecting the action would decrease the value, the probability of selecting this action would decrease

Instead of using this directly as loss function, to stabilize the implementation by adjusting the policy optimization step size, the loss is extended in a pessimistic way:

$$
loss_{actor} = \min [r_\Theta (s_t, a_t) A_\Phi (s_t, a_t), clip(r_\Theta (s_t, a_t), 1-\epsilon, 1+\epsilon) A_\Phi (s_t, a_t)]
$$

In [2]:
class Actor(tf.keras.Model):    #change inheritance class to enable saving the models
    def __init__(self, units=(400, 300), n_actions=2, **kwargs):
        super(Actor, self).__init__(**kwargs)
        self._layers = []
        for i, u in enumerate(units):
            self._layers.append(tf.keras.layers.Dense(u, activation='relu'))
        self._layers.append(tf.keras.layers.Dense(n_actions, activation='softmax'))
        
    def call(self, inputs, **kwargs):
        outputs = inputs
        for l in self._layers:
            outputs = l(outputs)
        return outputs
    
class Critic(tf.keras.Model):
    def __init__(self, units=(400, 300), **kwargs):
        super(Critic, self).__init__(**kwargs)
        self._layers = []
        for i, u in enumerate(units):
            self._layers.append(tf.keras.layers.Dense(u, activation='relu'))
        self._layers.append(tf.keras.layers.Dense(1))
        
    def call(self, inputs, **kwargs):
        outputs = inputs
        for l in self._layers:
            outputs = l(outputs)
        return outputs
    
class PPOAgent:
    def __init__(self, action_space, observation_space, gamma=0.99, epsilon = 0.1):
        self.gamma = gamma
        self.epsilon = epsilon
        self.action_space = action_space
        self.observation_shape = observation_space.shape[0]
        self.actor = Actor(n_actions=action_space.n)
        self.actor_old = Actor(n_actions=action_space.n)
        self.critic = Critic()
        self.target_critic = Critic()
        
        self.actor_learning_rate=0.00025
        self.critic_learning_rate=0.001
        self.actor_optimizer = tf.keras.optimizers.Adam(self.actor_learning_rate)
        self.critic_optimizer = tf.keras.optimizers.Adam(self.critic_learning_rate)   # default = 0,001 -> hatten wir auch schon
        self._init_networks()
        
    def _init_networks(self):
        initializer = np.zeros([1, self.observation_shape])   # ergänzt zu V1 -> hatten wir aber auch schon gemacht
        self.actor(initializer)
        self.actor_old(initializer)
        
        self.critic(initializer)
        self.target_critic(initializer)
        
        self.update_frozen_nets()
        
    def act(self, observation):
        probs = self.actor(observation).numpy()
        probs = np.squeeze(probs)
        action = np.random.choice(self.action_space.n, p=probs)
        return action
    
    def get_critic_grads(self, states, rewards, next_states, dones):
        with tf.GradientTape() as tape:
            next_value = self.target_critic(next_states)
            q_value = rewards + (1-dones) * self.gamma * next_value
            value = self.critic(states)
            
            advantage = q_value - value
            loss = tf.reduce_mean(tf.square(advantage))
        gradients = tape.gradient(loss, self.critic.trainable_variables)
        return gradients, loss, advantage
    
    def get_actor_grads(self, states, actions, advantage):
        with tf.GradientTape() as tape:
            p_current = tf.gather(self.actor(states), actions, axis=1)
            p_old = tf.gather(self.actor_old(states), actions, axis=1)
            ratio = p_current / p_old
            clip_ratio = tf.clip_by_value(ratio, 1-self.epsilon, 1+self.epsilon)
            # entropy loss hatten wir probiert, bringt aber wenig --> sollte eigentlich exploration förndern
            # standardize advantage
            advantage = (advantage - tf.reduce_mean(advantage)) / (tf.keras.backend.std(advantage) + 1e-8)
            objective = ratio * advantage
            clip_objective = clip_ratio * advantage
            loss = -tf.reduce_mean(tf.where(objective < clip_objective, objective, clip_objective))
        gradients = tape.gradient(loss, self.actor.trainable_variables)
        return gradients, loss
        

    def learn(self, states, actions, rewards, next_states, dones):
        critic_grads, critic_loss, advantage = self.get_critic_grads(states, rewards, next_states, dones)
        actor_grads, actor_loss = self.get_actor_grads(states, actions, advantage)
        
        self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_variables))
        self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_variables))
        return actor_loss, critic_loss
    
    def update_frozen_nets(self):
        # TODO: set discount factor  -> was soll der hier bringen?
        weights = self.actor.get_weights()
        self.actor_old.set_weights(weights)
        
        weights = self.critic.get_weights()
        self.target_critic.set_weights(weights)

    def save_models(self, actor_path='actor_weights.h5', critic_path='critic_weights.h5'):
        self.actor.save_weights(actor_path)
        self.critic.save_weights(critic_path)

    def load_models(self, actor_path='actor_weights.h5', critic_path='critic_weights.h5'):
        try:
            self.actor.load_weights(actor_path)
            self.critic.load_weights(critic_path)
            print('Model loaded sucessful')
        except Exception as e:
            print(f"Error: {e}")


## Training

PPO is an on-policy method. We allways complete a full episode, record the trajectory and the rewards. We then use these to update our network. 

In [3]:
def compute_avg_return(env, agent, num_episodes=1, max_steps=1000, render=False):
    total_return = 0.0
    for _ in range(num_episodes):
        obs, _ = env.reset()
        episode_return = 0.0
        done = False
        steps = 0
        while not (done or steps > max_steps):
            if render:
                clear_output(wait=True)
                plt.axis('off')
                plt.imshow(env.render())
                plt.show()
            action = agent.act(np.array([obs]))
            obs, r, termination, truncation, _ = env.step(action)
            if termination or truncation:
                done = True
            episode_return += r
            steps += 1
        total_return += episode_return
        #print('episode_return', episode_return)
    return total_return / num_episodes

## Add a Connection to Tensorboard -> online visualization

In [4]:
# refers to log data and model data -> below for model data
jetzt = datetime.datetime.now()
datum_uhrzeit = jetzt.strftime("%Y%m%d_%H%M%S")
savedir = f'model\\CustomMountainCarVelocity_mod_sampling_5-16-8_{datum_uhrzeit}'
os.makedirs('model', exist_ok=True)
os.makedirs(savedir, exist_ok=True)


In [5]:
log_dir = f"{savedir}\\log"
os.makedirs(log_dir, exist_ok=True)

if os.path.exists(log_dir):
    print(f"The directory {log_dir} exists.")
    absolute_path = os.path.abspath(log_dir)
    print(absolute_path)
else:
    print(f"The directory {log_dir} does not exist.")
summary_writer = tf.summary.create_file_writer(log_dir)

# define all metrics to log
def log_metrics(epoch, total_timesteps, critic_loss, actor_loss, episode_return, actor_learning_rate, critic_learning_rate, epsilon, gamma):
    with summary_writer.as_default():
        tf.summary.scalar('epoch', epoch, step=epoch)
        tf.summary.scalar('total_timesteps', total_timesteps, step=epoch)
        tf.summary.scalar('critic_loss', critic_loss, step=epoch)
        tf.summary.scalar('actor_loss', actor_loss, step=epoch)
        tf.summary.scalar('episode_return', episode_return, step=epoch)
        tf.summary.scalar('critic_learning_rate', critic_learning_rate, step=epoch)
        tf.summary.scalar('actor_learning_rate', actor_learning_rate, step=epoch)
        tf.summary.scalar('discount_factor_gamma', gamma, step=epoch)
        tf.summary.scalar('clip_range_epsilon', epsilon, step=epoch)

# tensorboard --logdir log in bash to open the online editor

The directory model\CustomMountainCarVelocity_mod_sampling_5-16-8_20240107_120820\log exists.
c:\#_FESTPLATTE\06_Studium\Master_HKA\Semesterdateien\Semester 1\Roboterprogrammierung-Hein\Projekt\Coding\model\CustomMountainCarVelocity_mod_sampling_5-16-8_20240107_120820\log


In [6]:
# wrap custom enviroment with changed reward strategies
class CustomMountainCarPosition(gym.Wrapper):       # Reward wird vergeben je höher car kommt
    def __init__(self, inheritance_env):
        super(CustomMountainCarPosition, self).__init__(inheritance_env)

    def step(self, action):

        observation, reward, terminated, truncated, info = self.env.step(action)    # "normale" step methode aufrufen
 
        x_position = observation[0]
        if x_position >= -0.5:
            exp = int((x_position + 0.5) * 10)  # Exponent berechnen (z.B. -0.4 -> 0, -0.3 -> 1, ..., 0,4 -> 8)
            reward = 2 ** exp
            #print("Reward:", reward)

        # elif x_position >= -0.6:                
        #     exp = int((-x_position - 0.6) * 10)  # Exponent berechnen (z.B. -0,6 -> 0, -0,7 -> 1, ..., -1,2 -> 6)
        #     reward = 2 ** exp

        return observation, reward, terminated, truncated, info

In [7]:
class CustomMountainCarVelocity(gym.Wrapper):             # Reward wird vergeben wenn velocity hoch ist
    def __init__(self, inheritance_env):
        super(CustomMountainCarVelocity, self).__init__(inheritance_env)

    def step(self, action):
        observation, reward, terminated, truncated, info = self.env.step(action)    # "normale" step methode aufrufen

        #print("Velocity: ", observation[1])
        if observation[1] < -0.02 or observation[1] > 0.02:
            reward = 2

        if observation[1] < -0.03 or observation[1] > 0.03:
            reward = 4

        if observation[1] < -0.04 or observation[1] > 0.04:
            reward = 8

        if observation[1] < -0.05 or observation[1] > 0.05:
            reward = 16

        if observation[1] < -0.06 or observation[1] > 0.06:
            reward = 32

        if terminated:
            reward = 1000
            #print("######## Terminated ########")

        return observation, reward, terminated, truncated, info

In [8]:
#env = gym.make('MountainCar-v0', render_mode='rgb_array')  #human fur pygame gui -> very laggy!

inheritance_env = gym.make("MountainCar-v0")
env = CustomMountainCarVelocity(inheritance_env)

agent = PPOAgent(env.action_space, env.observation_space)   # observation_space entpacken für obserbation_shape (siehe __init__)

In [9]:
# memics settings from stable baselines -> goal faster computing
n_rollouts = 5 
batch_size = 64
learn_steps = 8     
total_timesteps = 0
step_counter = 0

#goal = 600.000 timesteps -> baseline has learned properly here
for epoch in range(290):    # one epoch -> one complete learning iteration bzw. one update of the frozen nets
    
    states = []
    rewards = []
    actions = []
    next_states = []
    dones = []
    
    # collect experience in the enviroment with current policy for n episodes/ rollouts
    # give the agent more time to collect experiences more apart from the starting state
    for rollouts in range(n_rollouts):  
        obs, _ = env.reset()
        done = False  

        while not done:
            env.render()    # call gui if render_mode = 'human'
            action = agent.act(np.array([obs]))
            new_obs, r, termination, truncation, _ = env.step(action)
            if termination:
                print('######TERMINATED######')
            if termination or truncation:
                done = True

            states.append(obs)
            rewards.append([r])
            actions.append([action])
            obs = new_obs
            next_states.append(obs)
            dones.append([done])

            total_timesteps += 1
            step_counter += 1
    
    step_counter = 0
    # store colledted experience for all rollouts/ episodes and reset enviroment for next episode/ rollout
    states, actions, rewards, next_states, dones = map(np.array, [states, actions, rewards, next_states, dones])
    obs, _ = env.reset()
    #print('collected experience in n rollouts')

    # learn policy and value from the collected data 
    for _ in range(learn_steps):
        indices = np.arange(states.shape[0])
        np.random.shuffle(indices)  # create random indice row
        
        # switch indices to random experience distribution
        shuffled_states = states[indices]
        shuffled_actions = actions[indices]
        shuffled_rewards = rewards[indices]
        shuffled_next_states = next_states[indices]
        shuffled_dones = dones[indices]

        # divides the whole shuffled experience into batches of batch_size
        for j in range(0, states.shape[0], batch_size):
            states_batch = shuffled_states[j:j + batch_size]    # j:j + batch_size -> returns all elements from x*batch_size to (x+1)*batch_size
            actions_batch = shuffled_actions[j:j + batch_size]
            rewards_batch = shuffled_rewards[j:j + batch_size]
            next_states_batch = shuffled_next_states[j:j + batch_size]
            dones_batch = shuffled_dones[j:j + batch_size]

            # push one batch of the shuffled experience to the learning method -> one update of the current nets (actor and critic) per passed batch of experience
            actor_loss, critic_loss = agent.learn(states_batch,
                                                  actions_batch,
                                                  rewards_batch,
                                                  next_states_batch,
                                                  dones_batch)
            #print('update online nets, learn from one batch')
    agent.update_frozen_nets()
    #print('update frozen nets, do one learn step')
    one_episode_return = compute_avg_return(env, agent, num_episodes=1)
    print(f'epoch {epoch + 1}, total_timesteps {total_timesteps}, actor loss {actor_loss}, critic loss {critic_loss}, _return {one_episode_return}')
    # Log metrics at the end of each epoch
    log_metrics(epoch = epoch, total_timesteps = total_timesteps,
                critic_loss = critic_loss, actor_loss= actor_loss,
                episode_return = one_episode_return,
                actor_learning_rate = agent.actor_learning_rate, critic_learning_rate = agent.critic_learning_rate,
                epsilon = agent.epsilon, gamma = agent.gamma)
    
    # do prints every 10 epochs
    if (epoch + 1) % 10 == 0:
        avg_return = compute_avg_return(env, agent, num_episodes=1)
        print(f'epoch {epoch + 1}, total_timesteps {total_timesteps}, actor loss {actor_loss}, critic loss {critic_loss}, avg_return {avg_return}')
    
env.close() # kill gui

c:\Users\robin\anaconda3\Lib\site-packages\gymnasium\envs\classic_control\mountain_car.py:173: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("MountainCar-v0", render_mode="rgb_array")
  gym.logger.warn(


epoch 10, total_timesteps 10000, actor loss 0.003568469313904643, critic loss 0.0006245007971301675, avg_return -200.0
epoch 20, total_timesteps 20000, actor loss -0.04550672695040703, critic loss 0.025152314454317093, avg_return -200.0
epoch 30, total_timesteps 30000, actor loss -0.012108182534575462, critic loss 0.008307019248604774, avg_return -200.0
epoch 40, total_timesteps 40000, actor loss -0.028688006103038788, critic loss 0.21905425190925598, avg_return -200.0
epoch 50, total_timesteps 50000, actor loss -0.02108853869140148, critic loss 0.08991749584674835, avg_return -200.0
epoch 60, total_timesteps 60000, actor loss -0.025161752477288246, critic loss 0.030206799507141113, avg_return -200.0
epoch 70, total_timesteps 70000, actor loss -0.015246591530740261, critic loss 0.029146483168005943, avg_return -200.0
epoch 80, total_timesteps 80000, actor loss 0.014826688915491104, critic loss 52.51908493041992, avg_return -200.0
epoch 90, total_timesteps 90000, actor loss -0.006715798

# Storing and loading models

In [10]:
# save the model to h5 format
#filepath_actor = f"model/default_CartPole_{datum_uhrzeit}/actor.h5"
filepath_actor = f"{savedir}\\actor.h5"

#filepath_critic = f"model/default_CartPole_{datum_uhrzeit}/critic.h5"
filepath_critic = f"{savedir}\\critic.h5"

agent.save_models(filepath_actor, filepath_critic)

In [15]:
# load the model from h5 format -> use new agent in new instance of the enviroment to prevent overwriting
load_env = gym.make("MountainCar-v0", render_mode='rgb_array')

load_agent = PPOAgent(load_env.action_space, load_env.observation_space)
load_agent._init_networks()

filepath_actor = f"model\\CustomMountainCarVelocity_mod_sampling_5-16-8_20240107_120820\\actor.h5"
filepath_critic = f"model\\CustomMountainCarVelocity_mod_sampling_5-16-8_20240107_120820\\critic.h5"

load_agent.load_models(filepath_actor, filepath_critic)

Model loaded sucessful


render enviroment without pygame --> instead with matplotlib , a little bit laggy but ok

In [16]:
compute_avg_return(load_env, load_agent, num_episodes=20, render=True)
load_env.close()

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>